# Random Forest Regressor for Kaggle New York City Taxi Fare Prediction [competition](https://www.kaggle.com/c/new-york-city-taxi-fare-prediction)

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn import metrics
from sklearn.tree import export_graphviz
from sklearn.ensemble import forest

import matplotlib.pyplot as plt
import seaborn as sns

import IPython
from IPython.display import display
import graphviz

from datetime import datetime
import math
import re

import os

## The Data

The training data for this competition is 55M rows, each representing a taxi trip in New York City. Our goal is to use the data to provide a fare prediction to the rider based on the pickup time, number of passengers, and the pickup and dropoff locations. 

**NOTE:** Due to the constraints of the problem statement, we must be careful not to pull in external data that we would not reasonably expect to have access to at the beginning of the taxi ride in training our model.

### Import the Data

In [ ]:
PATH = "./" # relative path to our data
!dir {PATH}

In [ ]:
n = 100 # load every nth row into df_raw

%time df_raw = pd.read_csv(f'{PATH}train.csv', low_memory=False, skiprows=lambda i: i % n != 0)

### Initial data inspection

In [ ]:
df_raw.shape # this gives us about half a million rows, plent for quick prototyping

In [ ]:
df_raw.head()

In [ ]:
df_raw.info()

In [ ]:
df_raw.describe()

We immidiately see few problems with the data that need to be addressed:
1. The minimum fare_amount is -60. We expect it to be a positive number.
2. Our latitudes and longitudes for both pickupp and dropoff have some obviously non-sensical values at the extremes.
3. I'm reasonably certain a cab can't hold 218 passengers. Let's set a limit of 10.

Let's just get rid of these data points.

In [ ]:
print(f'Old size: {len(df_raw)}')

min_fare = 0
max_pass = 10
lat_range = [30, 50]
lon_range = [-85, -65]

df_raw = df_raw[(df_raw.pickup_latitude > lat_range[0]) & 
                (df_raw.pickup_latitude < lat_range[1]) & 
                (df_raw.pickup_longitude > lon_range[0]) & 
                (df_raw.pickup_longitude < lon_range[1]) &
                (df_raw.dropoff_latitude > lat_range[0]) & 
                (df_raw.dropoff_latitude < lat_range[1]) & 
                (df_raw.dropoff_longitude > lon_range[0]) & 
                (df_raw.dropoff_longitude < lon_range[1]) & 
                (df_raw.fare_amount > min_fare) & 
                (df_raw.passenger_count < max_pass)]
print(f'New size: {len(df_raw)}')

That took off a little over 10,000 entries!

In [ ]:
df_raw.describe()

## Pre-processing

### Distance feature engineering

In [ ]:
# create two new features representing the latitude and longitude vectors traversed during the trip
def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

add_travel_vector_features(df_raw)

In [ ]:
# distance – in units of degrees – travelled during each trip
def add_distance_feature(df):
    df['distance'] = np.sqrt(df.abs_diff_longitude**2 + df.abs_diff_latitude**2)
    
add_distance_feature(df_raw)

In [ ]:
plot = df_raw.iloc[:2000].plot.scatter('abs_diff_longitude', 'abs_diff_latitude', alpha=0.5, s=7)

Let's take a quick look at the distribution for both distance travelled and taxi fare within a limited range. These limits will help us better see the shape of the distribution.

In [ ]:
short_rides = df_raw[(df_raw['abs_diff_latitude'] < 0.1) & (df_raw['abs_diff_longitude'] < 0.1)]

# Kernel Density Plot for distance travelled 
fig = plt.figure(figsize=(15,4),)
ax=sns.kdeplot(short_rides.distance , color='steelblue',shade=True,label='distance')
plt.title('Taxi Ride Distance Distribution')

The distance travelled follows a positive skewed normal distribution, with the max frequency at less than 0.02 degrees (about 1.4 miles). This makes intuitive sense. We'd expect most rides in NYC to be short, likely within Manhattan, with a few longer rides.

In [ ]:
cheap_rides = df_raw[df_raw.fare_amount < 50]

# Kernel Density Plot for taxi fare
fig = plt.figure(figsize=(15,4),)
ax=sns.kdeplot(cheap_rides.fare_amount , color='red',shade=True,label='fare_amount')
plt.title('Taxi Ride Fare')

Not surprisingly, taxi fares follow a very similar distribution. We'd expect the two to be highly correlated.

In [ ]:
print("Correlation between taxi fare and distance travelled: "+
      f"{df_raw['fare_amount'].corr(df_raw['distance'])}")

We see that the taxi fare has a high positive correlation with distance travelled. Generally speaking, longer rides are likely to cost more. Shocker!

## Random Forests

### Quick Baseline Model

In [ ]:
X_train = df_raw.drop(['key', 'pickup_datetime', 'fare_amount'], axis=1)
y_train = df_raw.fare_amount

m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
m.score(X_train, y_train)

Our quick and dirty Random Forest model has an $R^2$ of 0.96. However, this isn't as impressive a feat as it might seem. First, let's see how it does on our test data.

In [ ]:
test_raw = pd.read_csv(f'{PATH}test.csv')
test_raw.head()

In [ ]:
test_raw.shape

In [ ]:
test_raw.describe()

Here we run into a problem. Our test data set doesn't actually contain the fare amounts and so we'd have to do one of two things:
1. Test our model by submitting to Kaggle. This will give us an RMSE, not an $R^2$.
2. Hold out a small portion of our training set as a validation set, retrain our model, and test it on the validation set.

Let's do the latter.

In [ ]:
def split_vals(df, n): return df[:n].copy(), df[n:].copy()

n_valid = 9914  # same as Kaggle's test set size
n_trn = len(X_train)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(X_train, n_trn)
y_train, y_valid = split_vals(y_train, n_trn)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
m = RandomForestRegressor()
m.fit(X_train, y_train)
m.score(X_train, y_train)

In [ ]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [ ]:
print_score(m)

While this simple, no-brainer Random Forest isn't doing a terrible job on our validation set, we could do better. The drop in $R^2$ from training to validation means that we are seriously overfitting our data. Also, if you think about it, an RMSE of \$4.31 on a taxi fare prediction is rather high!

To reiterate, we'd like to be able to beat our **baseline RF Regressor $R^2$ of 0.80**.

### Bagging

Our Random Forest already uses a technique called bagging of multiple trees to give us more generalizable results. To understand what this does, let's take a look at what a single tree with no bagging looks like.

#### Use a Subset
We don't really need all 500k rows at this stage and can afford to use a small subset of this data for prototyping

In [ ]:
df_sub = raw_train.sample(40000)
X_sub = df_sub.drop(['key', 'pickup_datetime', 'fare_amount'], axis=1)
y_sub = df_sub.fare_amount

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_sub, y_sub)
m.score(X_valid, y_valid)

Using a 40,000 row sample of our training set and testing it on the same validation set gives only a small drop in $R^2$. Let's rename our subsets so we can use the print_score function on them, which takes X_train and y_train.

In [ ]:
X_train = X_sub
y_train = y_sub

print_score(m)

#### Single Tree

In [ ]:
m = RandomForestRegressor(n_estimators=1, max_depth=3, bootstrap=False, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
def draw_tree(t, df, size=10, ratio=0.6, precision=0):
    s=export_graphviz(t, out_file=None, feature_names=df.columns, filled=True,
                      special_characters=True, rotate=True, precision=precision)
    IPython.display.display(graphviz.Source(re.sub('Tree {',
       f'Tree {{ size={size}; ratio={ratio}', s)))

In [ ]:
draw_tree(m.estimators_[0], X_train, precision=3)

That's not a bad $R^2$!

Most of our splits so far are on distance, telling us that from the data available, distance appears to be by some margin the most predictive of the features available. Engineering that feature appears to have been a good decision. While the RF would have made an insight along these lines by splitting on locations instead, having the distance feature available reduces the number of splits necessary.

To draw more complex insights, if they exist, we would have to do some more feature engineering.

#### Bagged Trees

First, let's go back to our baseline model, which is a forest of randomly bagged trees.

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
y_valid.values[0]

In [ ]:
preds = np.stack([t.predict(X_valid) for t in m.estimators_])
preds[:,0], np.mean(preds[:,0])

Each of our 10 trees – `sklearn` calls them etimators – makes its own (not great) prediction. The mean of these predictions, however, is close to much closer to our known fare value of 9.0 than the individual predictions themselves. This averaging of trees is called bagging.

In [ ]:
preds.shape

In [ ]:
plt.plot([metrics.r2_score(y_valid, np.mean(preds[:i+1], axis=0)) for i in range(10)]);

This plot suggests that increasing the number of trees beyond 8 or 10 won't help us much. Let's take a look for ourselves.

In [ ]:
m = RandomForestRegressor(n_estimators=20, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
m = RandomForestRegressor(n_estimators=100, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

That last jump from 40 to 100 estimators made almost no difference for the accuracy of our model.

**NOTE**: You may see slightly different numbers when you run this. That's a lesson to me not to randomize how I sample my data. While not always the case, you will likely see some small increase in R^2 as you increase `n_estimators`

### Engineering Datetime Features

One of our fields is the pickup_datetime. You may have noticed that I removed this from the dataset before passing it to our model. I shall now train the same RF with the pickup_datetime. Then, I'll engineer some features from the information that this field provides us to see if that improves the model even further.

In [ ]:
X_train = df_raw.drop(['key', 'fare_amount'], axis=1)
y_train = df_raw.fare_amount

n_valid = 9914  # same as Kaggle's test set size
n_trn = len(X_train)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(X_train, n_trn)
y_train, y_valid = split_vals(y_train, n_trn)

# We should now have 9 cols instead of our earlier 8.
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
raw_train = raw_train.sample(40000)
X_train = raw_train.drop(['key', 'fare_amount'], axis=1)
y_train = raw_train.fare_amount

X_train.shape, y_train.shape

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

We realize now that, in fact, the Random Forest cannot even use the `pickup_datetime` field in this form since it is a string. This field is of no use to us without some feature engineering. Let's do that and then remove the pickup_datetime field from our dataset.

In [ ]:
def add_datetime_features(df):
    
    year = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S %Z" ).year
    df['year'] = df['pickup_datetime'].map(year)
    print('1/7')
    
    hour = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S %Z" ).hour
    df['hour'] = df['pickup_datetime'].map(hour)
    print('2/7')

    day_of_week = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S %Z" ).weekday()
    df['day_of_week'] = df['pickup_datetime'].map(day_of_week)
    print('3/7')

    month = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S %Z" ).month
    df['month'] = df['pickup_datetime'].map(month)
    print('4/7')

    week_number = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S %Z" ).strftime('%V')
    df['week_number'] = df['pickup_datetime'].map(week_number)
    print('5/7')
    
    seasons = [0,0,1,1,1,2,2,2,3,3,3,0] #dec - feb is winter, then spring, summer, fall etc
    season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d %H:%M:%S %Z" ).month-1)]
    df['season'] = df['pickup_datetime'].map(season)
    print('6/7')
    
    # 10pm-5am is late-night
    late_night_hours = [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
    late_night = lambda x: late_night_hours[(datetime.strptime(x, "%Y-%m-%d %H:%M:%S %Z" ).hour)]
    df['late_night'] = df['pickup_datetime'].map(late_night)
    print('7/7. All done!')
    
add_datetime_features(df_raw)

In [ ]:
df_raw.head()

In [ ]:
X_train = df_raw.drop(['key', 'fare_amount', 'pickup_datetime'], axis=1)
y_train = df_raw.fare_amount

X_train, X_valid = split_vals(X_train, n_trn)
y_train, y_valid = split_vals(y_train, n_trn)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
df_sub = df_raw.sample(40000)

In [ ]:
X_train = df_sub.drop(['key', 'pickup_datetime', 'fare_amount'], axis=1)
y_train = df_sub.fare_amount

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

While we're doing noticeably better in terms of validation R^2, our biggest problem is still that we're overfitting drastically on the training data. The model simply isn't generalizing. 

I borrowed a function called `set_rf_samples` from the fastai library to help combat this. Instead of sampling our training data up-front like scikit-learn does and then averaging together several `estimators` trained on that data, we could choose a different random subsample for each tree. This way, they don't have the opportunity to fit the whole training set as closely.

In [ ]:
def set_rf_samples(n):
    """ Changes Scikit learn's random forests to give each tree a random sample of
    n random rows.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n))

In [ ]:
def reset_rf_samples():
    """ Undoes the changes produced by set_rf_samples.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n_samples))

In [ ]:
df_raw.shape

We'll have to return to our full dataset to see this technique in action.

In [ ]:
X_train = df_raw.drop(['key', 'fare_amount', 'pickup_datetime'], axis=1)
y_train = df_raw.fare_amount

X_train, X_valid = split_vals(X_train, n_trn)
y_train, y_valid = split_vals(y_train, n_trn)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
set_rf_samples(20000)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

In [ ]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

You'll notice that our model performs about the same on the training and validation data now, a sign isn't overfit and that it generalizes well to new observations. What's more, we didn't lose any predictive power in the process and still have an R^2 of 0.84

However, the most impressive part is that we did all this without any hyperparameter tuning! A `RandomForestRegressor` with all the defaults (coupled with a few techniques to ensure our model generalizes and some feature engineering) can predict the fare of a taxi using not much more than pickup and dropoff locations and pickup time. It does so with an RMSE of less than 4 dollars, which is not too bad considering our mean fare amount is over 11 dollars.

A real-world model to do something like that is likely to gain a fair bit of predictive accuracy from access to real time traffic flow and routing information. While walking through the hyperparameter tuning process is beyond the scope of this notebook, suffice it to say for now that tweaking some of these defaults can increase performance a little. I will however change the number of subsamples we choose for each tree to see if that helps our accuracy.

In [ ]:
set_rf_samples(50000)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

It did! This process might seem like a bit of trial and error, and it is. That said, if you're interested, `sklearn.model_selection.RandomizedSearchCV` and `sklearn.model_Selection.GridSearchCV` are useful tools to help automate this process to varying degrees (possibly at the cost of extra computational expense).

In [ ]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

Having trained a model with R^2 about 0.05 greater than our baseline and a validation RMSE that's better by just over 60c, let's get predictions on the test data and write them to a submission file that meets Kaaggle's specifications for the competition.

In [ ]:
test_raw = pd.read_csv(f'{PATH}test.csv')

In [ ]:
test_raw.shape
test_raw.head()

In [ ]:
add_travel_vector_features(test_raw)
add_distance_feature(test_raw)
add_datetime_features(test_raw)

test_raw.head()

In [ ]:
X_test = test_raw.drop(['pickup_datetime', 'key'], axis=1)
X_test.head()

In [ ]:
X_test.shape

In [ ]:
y_pred = m.predict(X_test)

In [ ]:
# Write the predictions to a CSV file which we can submit to the competition.
RF_submission2 = pd.DataFrame(
    {'key': test_raw.key, 'fare_amount': y_pred},
    columns = ['key', 'fare_amount'])
RF_submission2.to_csv('RF_submission.csv', index = False)

print(os.listdir('.'))